# RAG Chatbot Service

Gradio를 이용한 한국어 RAG 챗봇 서비스입니다.
- RAGChain은 최초 1회만 초기화됩니다.
- 이후로는 객체를 유지하면서 질문에 대해 빠르게 답변합니다.
- 검색 방식(retrieval_type)과 하이브리드 가중치(hybrid_alpha)를 조정할 수 있습니다.

In [1]:
import sys
from pathlib import Path

# Add project path to sys.path
project_path = str(Path('.').resolve().parent)
if project_path not in sys.path:
    sys.path.append(project_path)

import gradio as gr
from pipeline.chain.generation import RAGChain
from pipeline.common import setup_logger
from pipeline.eval.utils import answer_extractor
from typing import Optional, Tuple
import time

# Global variables for RAGChain instance and current settings
rag_chain: Optional[RAGChain] = None
current_config = {
    'retrieval_type': 'hybrid',
    'hybrid_alpha': 0.6
}
logger = setup_logger("chatbot_logger")

print("✓ 라이브러리 로드 완료")

/venv/final_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ 라이브러리 로드 완료


In [2]:
def initialize_rag_chain(retrieval_type: str, hybrid_alpha: float) -> str:
    """
    RAGChain 초기화 함수
    - 처음 호출 시 RAGChain을 생성하고 전역 변수에 저장
    - 이후 호출 시 옵션이 변경되었으면 재초기화
    
    Args:
        retrieval_type: 'sparse', 'dense', 'hybrid' 중 하나
        hybrid_alpha: 하이브리드 검색의 BM25 가중치 (0.0~1.0)
    
    Returns:
        초기화 상태 메시지
    """
    global rag_chain, current_config, logger

    # 설정 업데이트
    current_config['retrieval_type'] = retrieval_type
    current_config['hybrid_alpha'] = hybrid_alpha

    # 설정이 바뀌었을 수 있으므로 객체 내 해당 변수들을 업데이트
    if rag_chain is not None:
        rag_chain.retrieval_type = retrieval_type
        rag_chain.hybrid_alpha = hybrid_alpha

        # 이미 초기화되어 있으므로 재사용
        msg = f"✓ 기존 RAGChain 재사용\n- 검색 방식: {retrieval_type}\n- 하이브리드 가중치: {hybrid_alpha}"
        logger.info(msg)
        return msg 
    
    try:
        msg = f"🔄 RAGChain 초기화 중...\n- 검색 방식: {retrieval_type}\n- 하이브리드 가중치: {hybrid_alpha}"
        logger.info(msg)
        print(msg)
        
        start_time = time.time()
        
        # RAGChain 초기화 (시간이 걸림)
        rag_chain = RAGChain(
            retrieval_type=retrieval_type,
            hybrid_alpha=hybrid_alpha,
            encoder_name='bge',
            chunk_size=1000,
            overlap_size=200,
            generator_name='midm',
            seq_type='dq',
            doc_rearrange=1,
            top_k=5,
            generator_type="vllm",
            vllm_api_base="http://localhost:8000/v1",
            with_retrieval_results=True,
            logger=logger
        )
        
        elapsed_time = time.time() - start_time
        success_msg = f"✓ RAGChain 초기화 완료 ({elapsed_time:.1f}초)\n- 검색 방식: {retrieval_type}\n- 하이브리드 가중치: {hybrid_alpha}"
        logger.info(success_msg)
        return success_msg
        
    except Exception as e:
        error_msg = f"✗ RAGChain 초기화 실패: {str(e)}"
        logger.error(error_msg)
        return error_msg

In [3]:
def answer_question(question: str, retrieval_type: str, hybrid_alpha: float) -> Tuple[str, str]:
    """
    사용자 질문에 대한 답변 생성 함수
    - RAGChain이 준비되지 않았으면 초기화
    - RAGChain의 .ask() 메서드로 답변 생성
    
    Args:
        question: 사용자 질문
        retrieval_type: 검색 방식
        hybrid_alpha: 하이브리드 가중치
    
    Returns:
        (답변, 검색된 문서 정보)
    """
    global rag_chain, logger
    
    if not question.strip():
        return "질문을 입력해주세요.", ""
    
    try:
        # RAGChain 초기화 (필요한 경우만)
        init_msg = initialize_rag_chain(retrieval_type, hybrid_alpha)
        
        if rag_chain is None:
            return "RAGChain 초기화 실패. 서버 설정을 확인해주세요.", ""
        
        logger.info(f"질문: {question}")
        start_time = time.time()
        
        # RAGChain의 .ask() 메서드 호출 (빠름: 1-2초)
        result = rag_chain.ask(question)
        
        elapsed_time = time.time() - start_time
        
        # 결과 처리
        if isinstance(result, list) and len(result) > 0:
            answer, retrieved_docs = result[0]
            
            # answer_extractor를 사용하여 실제 답변 텍스트만 추출
            extracted_answer = answer_extractor(answer)
            
            # 검색된 문서 정보 포맷팅 (문서 내용만)
            doc_info = f"**검색된 상위 문서**:\n\n"
            for i, doc in enumerate(retrieved_docs):
                doc_info += f"**{i}.** {doc['chunked_text']}...\n\n"
            
            logger.info(f"답변 생성 완료 (소요시간: {elapsed_time:.2f}초)")
            return extracted_answer, doc_info
        else:
            return "답변 생성 실패", ""
            
    except Exception as e:
        error_msg = f"오류 발생: {str(e)}"
        logger.error(error_msg)
        return error_msg, ""

In [4]:
def create_gradio_interface():
    """
    Gradio 인터페이스 생성 및 반환
    """
    with gr.Blocks(title="RAG 챗봇 서비스", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🤖 RAG 기반 한국어 챗봇 서비스")
        gr.Markdown("뉴스 기사 기반 검색증강생성(RAG) 챗봇입니다. 질문을 입력하면 관련 문서를 검색한 후 답변을 생성합니다.")
        
        with gr.Row():
            with gr.Column(scale=3):
                question_input = gr.Textbox(
                    label="질문을 입력하세요",
                    placeholder="예: 최근 정치 뉴스는 어떻게 돼?",
                    lines=3
                )
                ask_button = gr.Button("🔍 답변 생성", variant="primary", size="lg")
            
            with gr.Column(scale=1):
                gr.Markdown("### ⚙️ 검색 옵션")
                retrieval_type_select = gr.Radio(
                    choices=["sparse", "dense", "hybrid"],
                    value="hybrid",
                    label="검색 방식",
                    info="BM25(sparse) / 벡터 검색(dense) / 혼합(hybrid)"
                )
                hybrid_alpha_slider = gr.Slider(
                    minimum=0.0,
                    maximum=1.0,
                    value=0.6,
                    step=0.1,
                    label="혼합 검색 가중치",
                    info="0(BM25만) ~ 1(벡터 검색)",
                    visible=True
                )
        
        # 옵션 가시성 제어
        def update_alpha_visibility(retrieval_type):
            return gr.update(visible=(retrieval_type == "hybrid"))
        
        retrieval_type_select.change(
            update_alpha_visibility,
            inputs=retrieval_type_select,
            outputs=hybrid_alpha_slider
        )
        
        with gr.Row():
            with gr.Column():
                answer_output = gr.Markdown(
                    label="답변",
                    value="💬 질문을 입력하면 여기에 답변이 표시됩니다."
                )
            with gr.Column():
                docs_output = gr.Markdown(
                    label="검색된 문서",
                    value="📄 검색된 문서 정보가 여기에 표시됩니다."
                )
        
        with gr.Row():
            status_output = gr.Textbox(
                label="상태",
                interactive=False,
                value="준비 완료"
            )
        
        # 버튼 클릭 이벤트
        def on_ask_click(question, retrieval_type, hybrid_alpha):
            status_output.value = "처리 중..."
            
            # retrieval_type이 hybrid가 아니면 hybrid_alpha는 무시
            if retrieval_type != "hybrid":
                hybrid_alpha = 0.6  # default value
            
            answer, docs = answer_question(question, retrieval_type, hybrid_alpha)
            
            status_output.value = "완료"
            return answer, docs, status_output.value
        
        ask_button.click(
            on_ask_click,
            inputs=[question_input, retrieval_type_select, hybrid_alpha_slider],
            outputs=[answer_output, docs_output, status_output]
        )
        
        # Enter 키로도 질문 제출 가능
        question_input.submit(
            on_ask_click,
            inputs=[question_input, retrieval_type_select, hybrid_alpha_slider],
            outputs=[answer_output, docs_output, status_output]
        )
    
    return demo

# Gradio 인터페이스 생성
demo = create_gradio_interface()

print("✓ Gradio 인터페이스 생성 완료")

/tmp/ipykernel_487983/2236825600.py:5: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(title="RAG 챗봇 서비스", theme=gr.themes.Soft()) as demo:


✓ Gradio 인터페이스 생성 완료


In [ ]:
# 서비스 실행
if __name__ == "__main__":
    print("\n" + "="*60)
    print("🚀 RAG 챗봇 서비스 시작")
    print("="*60)
    print("\n📝 사용 방법:")
    print("1. 검색 옵션을 선택합니다 (기본값: hybrid)")
    print("2. 질문을 입력합니다")
    print("3. '답변 생성' 버튼을 클릭하거나 Enter를 누릅니다")
    print("\n⚠️  주의사항:")
    print("- 최초 실행 시 RAGChain 초기화에 시간이 걸립니다")
    print("- 이후 질문은 빠르게 응답됩니다")
    print("\n" + "="*60 + "\n")
    
    demo.launch(share=True)


🚀 RAG 챗봇 서비스 시작

📝 사용 방법:
1. 검색 옵션을 선택합니다 (기본값: hybrid)
2. 질문을 입력합니다
3. '답변 생성' 버튼을 클릭하거나 Enter를 누릅니다

⚠️  주의사항:
- 최초 실행 시 RAGChain 초기화에 시간이 걸립니다
- 이후 질문은 빠르게 응답됩니다


* Running on local URL:  http://127.0.0.1:7860


* Running on public URL: https://f3401fcc69ab60089b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🔄 RAGChain 초기화 중...
- 검색 방식: hybrid
- 하이브리드 가중치: 0.6


[2025-12-21 05:40:55] INFO SentenceTransformer.py:219: Use pytorch device_name: cuda:0
[2025-12-21 05:40:55] INFO SentenceTransformer.py:227: Load pretrained SentenceTransformer: dragonkue/bge-m3-ko


Model is on device: cuda:0


Batches: 100%|██████████| 1/1 [00:00<00:00, 56.84it/s]
